In [1]:
# Install required libraries
#pip install pdf2image pytesseract opencv-python ollama pandas

     ---------------------------------------- 66.7/66.7 MB 3.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 45.5/45.5 kB 2.4 MB/s eta 0:00:00
Failed to build llama-cpp-python


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [20 lines of output]
  *** scikit-build-core 0.10.7 using CMake 3.31.4 (wheel)
  *** Configuring CMake...
  2025-02-04 23:13:00,516 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
  loading initial cache file C:\Users\melod\AppData\Local\Temp\tmpwsx0pt38\build\CMakeInit.txt
  -- Building for: NMake Makefiles
  CMake Error at CMakeLists.txt:3 (project):
    Running
  
     'nmake' '-?'
  
    failed with:
  
     no such file or directory
  
  
  CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
  CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
  -- Configuring incomplete, errors occurred!
  
  *** CMake configuration failed
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
ERROR: Could not build wheels for llama-cpp-python,

In [12]:
import pytesseract
from pdf2image import convert_from_path
import cv2
import numpy as np
import pandas as pd
import json
import ollama
from IPython.display import display, Markdown
import ipywidgets as widgets
import os

In [13]:
# Convert PDF to images
def pdf_to_images(pdf_path):
    return convert_from_path(pdf_path, dpi=300)

In [14]:
# Extract text using Tesseract OCR
def extract_text_from_images(images):
    extracted_text = ""
    for img in images:
        text = pytesseract.image_to_string(img)
        extracted_text += text + "\n"
    return extracted_text

In [15]:
# Extract tables using OpenCV & Tesseract
def extract_tables_from_images(images):
    tables = []
    for img in images:
        gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        table_text = pytesseract.image_to_string(thresh, config="--psm 6")

        # Convert text into structured table format
        table_lines = [line.split() for line in table_text.split("\n") if line.strip()]
        if table_lines:
            df = pd.DataFrame(table_lines)
            tables.append(df)

    return tables

In [16]:
# Process PDF and extract structured data
def process_pdf(pdf_path):
    images = pdf_to_images(pdf_path)
    text = extract_text_from_images(images)
    tables = extract_tables_from_images(images)
    return {"text": text, "tables": tables}

In [23]:
#Identification Layer
def identLayer(text, table_data, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Extracted Information From The Document is:
    {text}

    Extracted Table Data:
    {json.dumps([table.to_dict() for table in table_data], indent=2)}

    Fetch table line item "{table_item}" and identify its trend over the specified period in the document.
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [24]:
def getIdentLayerResponse(pdf_path, table_item):
    data = process_pdf(pdf_path)
    response = infoExtLayer(data["text"], data["tables"], table_item)
    return response

In [25]:
#Information Extraction Layer
def infoExtLayer(text, table_data, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Extracted Information From The Document is:
    {text}

    Extracted Table Data:
    {json.dumps([table.to_dict() for table in table_data], indent=2)}

    Focus only on the specific table line item: "{table_item}".
    - Identify this item in the Income Statement Summary table.
    - Collect all the related information related to "{table_item}" in the extracted text.
    - Provide all the structured facts and insights from the document and respond concisely.
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [26]:
def getInfoExtLayerResponse(pdf_path, table_item):
    data = process_pdf(pdf_path)
    response = infoExtLayer(data["text"], data["tables"], table_item)
    return response

In [27]:
#Reasoning/Implication Layer
def reasLayer(rules, text, table_item):
    prompt = f"""
    You are an AI analyst that extracts insights by reasoning over structured data.

    Given below are the facts and insights gathered from the given document about the line item {table_item}.
    {text}
    
    Here are a few factors that have an impact on the item: {table_item} :
    
    {rules}

    Focus only on the specific table line item: "{table_item}" and using the rules :
    - Identify the factors affecting {table_item} and correlate a few references of those factors from the text.
    - Pick up cause-effect relationships from the text corresponding to the factors present in the rules.
    - Collect a set of implications or reasons that attributed to the observed change in {table_item}.
    - List them concisely with quoted reference to statistical data and informational evidence from the text.
    """

    response = ollama.chat(model="llama3", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

In [21]:
def getReasLayerResponse(rules, text, table_item):
    response = reasLayer(rules, text, table_item)
    return response

In [ ]:
trend = getIdentLayerResponse("2023-annual-report.pdf", "Net Interest Income")

In [28]:
trend

Net Interest Income increased from **$35,779 million in 2021** to **$44,950 million in 2022 (26%)**, and to **$52,375 million in 2023 (17%)**.


In [ ]:
layer1_response = getInfoExtLayerResponse("2023-annual-report.pdf", "Net Interest Income")

In [32]:
layer1_response

### **Net Interest Income - Insights and Structured Facts from the 2023 Annual Report**

#### **Definition and Overview:**
Net Interest Income (NII) is the interest earned on debt securities, loans (including yield-related loan fees), and other interest-earning assets minus the interest paid on deposits, short-term borrowings, and long-term debt.

#### **Financial Performance in 2023:**
- **Net Interest Income (NII) for 2023**: $52.38 billion (compared to $44.95 billion in 2022).
- **Increase in NII**: **$7.4 billion** driven by higher interest rates, leading to increased income from interest-earning assets.
- **Net Interest Margin**: Increased from **2.63% in 2022 to 3.06% in 2023**.

#### **Breakdown of Interest Income and Expense:**
- **Interest income rose by $31.08 billion**, mainly due to higher loan yields and securities income.
- **Interest expense increased by $23.67 billion**, driven by higher rates on deposits and long-term debt.

#### **Segment Contributions to Net Interest

In [ ]:
rules = """
Key Factors Affecting NII:
- The mix and overall size of the earning assets portfolio.
- The cost of funding those assets.
- Variable sources of interest income, such as loan fees, periodic dividends, and collection of interest on nonaccrual loans.
- Changes in interest rates, impacting both income from assets and expenses on interest-bearing deposits and debt.

"""

In [34]:
getReasLayerResponse(rules, layer1_response, trend)


#### **Insights and Implications:**

The increase in **Net Interest Income (NII) from $44,950 million in 2022 to $52,375 million in 2023 (17%)** was driven by:  


1. **Interest Rate Sensitivity**: The bank's NII is highly sensitive to interest rate movements. The rise in interest rates on **loans and debt securities** not only significantly benefited NII but also increased funding costs, thus contributing to a **$7.4 billion increase in NII**.
2. **Deposit Costs Rising**: Higher interest rates led to an increase in expenses on interest-bearing deposits, impacting overall profitability.
3. **Earning Assets Growth:** The **loan portfolio grew**, particularly in **commercial and consumer loans**, leading to higher interest income.
4. **Loan Portfolio Impact**: Higher yields on loans contributed positively, but fluctuations in the loan mix can impact future NII performance.
5. **Higher Loan Yields:** Yields on loans and securities improved, **boosting income from variable sources such as

In [35]:
##JPMC

In [37]:
trend = getIdentLayerResponse("annualreport-2023_jpmc.pdf", "Return on Common Equity")

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?